# Tarea 2: NLP, RNNs y Transformer

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import re

import numpy as np              # Procesamiento numérico
import pandas as pd             # Manipulación de conjuntos de datos
import matplotlib.pyplot as plt # Gráficos

import matplotlib.ticker as ticker

from tqdm import tqdm

from torch.utils.data import DataLoader, Subset

import torch
import torch.nn as nn
import torch.nn.functional as F

## Carga del dataset

Para la tarea se utilizará el dataset IMDB, el cual consiste en reseñas de películas con sus respectivas etiquetas de si es una reseña positiva o negativa.

El dataset se encuentra disponible en HuggingFace, asi que se puede cargar directamente.

**Nota:** Para descargar el dataset se tiene que instalar previamente la librería "datasets"

In [3]:
# # installing datasets package
# !pip install datasets

In [3]:
from datasets import load_dataset

c:\Users\Bruno\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("stanfordnlp/imdb")

In [5]:
data_train = pd.DataFrame(dataset["train"])
data_test = pd.DataFrame(dataset["test"])

# shuffle datasets

data_train = data_train.sample(frac=1).reset_index(drop=True)
data_test = data_test.sample(frac=1).reset_index(drop=True)

data_train.head()

,text,label
0,"Seeing this movie, as I just did for the first...",0
1,Welcome to Collinwood is one of the most delig...,1
2,In A Woman Under the Influence Mabel goes craz...,0
3,"this movie is, despite its ""independent"" statu...",0
4,"Boris and Bela do well together in this film,w...",1


In [6]:
# fraction dataset

fraction = 0.1
data_train = data_train.sample(frac=fraction).reset_index(drop=True)
data_test = data_test.sample(frac=fraction).reset_index(drop=True)

In [7]:
# tokenization
from nltk.tokenize import word_tokenize
import nltk
from bs4 import BeautifulSoup

nltk.download('punkt')

token_data_train = data_train.copy()
token_data_test = data_test.copy()

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text(separator=" ")
    # Remove special characters and multiple spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

token_data_train["text"] = token_data_train["text"].apply(clean_text)
token_data_test["text"] = token_data_test["text"].apply(clean_text)

token_data_train["text"] = token_data_train["text"].apply(word_tokenize)
token_data_test["text"] = token_data_test["text"].apply(word_tokenize)

token_data_train.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\Bruno\AppData\Local\Temp\ipykernel_10336\4139184961.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text(separator=" ")


,text,label
0,"[The, movie, starts, out, with, some, scrollin...",0
1,"[This, film, is, about, a, group, of, five, fr...",0
2,"[I, first, saw, Jake, Gyllenhaal, in, Jarhead,...",1
3,"[I, tried, to, remove, anything, that, might, ...",0
4,"[One, of, the, most, important, artistic, move...",1


In [8]:
# vocabulary
from collections import Counter

vocab = Counter()

for text in token_data_train["text"]:
    vocab.update(text)

# most common words
print(vocab.most_common(10))

# vocab size
vocabulary_size = len(vocab)
num_embeddings = vocabulary_size + 2
print(f"Vocabulary size: {vocabulary_size}")

[('the', 28584), ('a', 15639), ('and', 15471), ('of', 14340), ('to', 13100), ('is', 10618), ('in', 8708), ('I', 8040), ('it', 8015), ('that', 6971)]
Vocabulary size: 32833


In [9]:
# word to index
word_to_index = {word: index for index, word in enumerate(vocab, 2)}
word_to_index["<PAD>"] = 0
word_to_index["<UNK>"] = 1

# index to word
index_to_word = {index: word for word, index in word_to_index.items()}
index_to_word[0] = "<PAD>"
index_to_word[1] = "<UNK>"

# encode text
def encode_text(text):
    return [word_to_index.get(word, 1) for word in text]

token_data_train["text"] = token_data_train["text"].apply(encode_text)
token_data_test["text"] = token_data_test["text"].apply(encode_text)

token_data_train.head()

,text,label
0,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",0
1,"[25, 99, 22, 46, 36, 100, 20, 13, 101, 82, 102...",0
2,"[75, 146, 224, 225, 226, 48, 227, 228, 36, 45,...",1
3,"[75, 338, 55, 339, 340, 43, 67, 127, 341, 36, ...",0
4,"[105, 20, 17, 204, 682, 683, 684, 48, 17, 685,...",1


In [10]:
# padding

max_length = np.max(token_data_train["text"].apply(len))

def pad_text(text):
    return text[:max_length] if len(text) > max_length else text + [0] * (max_length - len(text))

token_data_train["text"] = token_data_train["text"].apply(pad_text)
token_data_test["text"] = token_data_test["text"].apply(pad_text)

token_data_train.head()

,text,label
0,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",0
1,"[25, 99, 22, 46, 36, 100, 20, 13, 101, 82, 102...",0
2,"[75, 146, 224, 225, 226, 48, 227, 228, 36, 45,...",1
3,"[75, 338, 55, 339, 340, 43, 67, 127, 341, 36, ...",0
4,"[105, 20, 17, 204, 682, 683, 684, 48, 17, 685,...",1


In [11]:
# create validation set

validation_size = 0.2
validation_samples = int(len(token_data_train) * validation_size)

final_val = token_data_train[:validation_samples]
final_train = token_data_train[validation_samples:]
final_test = token_data_test

In [12]:
final_train.reset_index(drop=True, inplace=True)
final_train.head()

,text,label
0,"[83, 17, 3113, 34, 80, 435, 50, 3, 52, 214, 15...",0
1,"[2, 13530, 351, 20, 702, 7753, 7754, 11667, 25...",0
2,"[25, 2214, 7172, 4528, 955, 27, 3060, 17, 1353...",0
3,"[75, 1567, 84, 3421, 89, 85, 2121, 7362, 7363,...",0
4,"[865, 21, 17, 2567, 955, 13587, 13588, 131, 13...",1


 Por motivos de costo computacional y tiempo de entrenamiento, para la tarea pueden considerar una proporción del dataset,  por ejemplo el 10%. Pero tenga en consideración que si el tamaño del dataset es muy pequeño el rendimiento del modelo se verá afectado negativamente.

In [13]:
# PyTorch datasets

from torch.utils.data import Dataset

class IMDBDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return torch.tensor(self.data["text"][index]), torch.tensor(self.data["label"][index])
    
train_dataset = IMDBDataset(final_train)
val_dataset = IMDBDataset(final_val)
test_dataset = IMDBDataset(final_test)

# Report split sizes
print('Training set has {} instances'.format(len(final_train)))
print('Validation set has {} instances'.format(len(final_val)))
print('Test set has {} instances'.format(len(final_test)))

# PyTorch dataloaders

batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Training set has 2000 instances
Validation set has 500 instances
Test set has 2500 instances


In [15]:
# # NO

# import random

# # percentage
# ratio = 0.5
# dataset_size = len(token_data_train)
# train_size = int(dataset_size * ratio * 0.8)
# val_size = int(dataset_size * ratio * 0.2)
# test_size = int(dataset_size * ratio)

# train_subset = Subset(token_data_train, np.arange(0, train_size))
# val_subset = Subset(token_data_train, np.arange(train_size, train_size + val_size))
# test_subset = Subset(token_data_test, np.arange(test_size))

## Parte 1: Redes Neuronales Recurrentes (RNNs)

### Actividad 1

Responda las siguientes preguntas:


*   ¿Cuáles son las principales características de las redes neuronales recurrentes?

    **Respuesta:** Las redes neuronales recurrentes, como su nombre indica, poseen conexiones recurrentes
    lo que permite que la red mantenga una memoria de los datos que ha procesado previamente,
    de forma que el _output_ en un estado particual esta influenciado no solo por su _input_ actual
    sino también por los _inputs_ previos.
    
    Las RNN tienen la capacidad de recibir _inputs_ de longitud variable y producir _outputs_ de longitud variable.

*   ¿Cuáles son las desventajas y limitaciones de las redes RNNs?

    **Respuesta:** Las RNNs sufren del problema de desvanecimiento y explosión del gradiente, lo que
    dificulta el entrenamiento de la red. Además, las RNNs tienen dificultades para mantener
    la memoria a largo plazo, lo que limita su capacidad para recordar información relevante
    de _inputs_ muy antiguos. Por el contrario, las RNNs también tienen problemas para olvidar
    información irrelevantes de _inputs_ recientes que no son relevantes para el _output_.

*   Menciona y describa brevemente al menos dos variantes a la arquitectura de RNNs simples.

    **Respuesta:** LSTM: Las redes LSTM (Long Short-Term Memory) son una variante de las RNNs que
    intentan solucionar el problema de la memoria a largo plazo. Las LSTM poseen una estructura
    más compleja que las RNNs tradicionales, con una estructura de celdas que permiten mantener
    la memoria a largo plazo y olvidar información irrelevante.

    GRU: Las redes GRU (Gated Recurrent Unit) son una variante de las RNNs que intentan simplificar
    la estructura de las LSTM, eliminando la estructura de celdas y manteniendo solo las puertas
    de olvido y actualización. Las GRU son más simples que las LSTM y tienen menos parámetros,
    lo que las hace más fáciles de entrenar y menos propensas al sobreajuste.

    Modelos de atención: Los modelos de atención son una variante de las RNNs que permiten
    que la red se enfoque en partes específicas de la secuencia de _inputs_ para generar el _output_,
    asignando pesos a los _inputs_ en vez de asignar el mismo peso a todos los _inputs_ como lo hacen
    las RNNs tradicionales. 

### Actividad 2

Implemente una red RNN simple y realice su entrenamiento con el dataset preprocesado. Para el pre-procesamiento use el método de la Actividad 1.

La siguiente celda contiene el esqueleto de la red. Puede usarlo como referencia para su implementación o realizar una propia.

In [14]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size

        # self.i2h = nn.Linear(vocab_size, hidden_size)
        self.i2e = nn.Embedding(vocab_size, embedding_size)
        self.e2h = nn.Linear(embedding_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        # hidden = self.i2h(input)
        input = self.i2e(input)
        hidden = self.e2h(input) + self.h2h(hidden)
        hidden = torch.tanh(hidden)
        output = self.h2o(hidden[:, -1, :]) # last hidden state
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [15]:
simple_model = RNN(num_embeddings, 128, 128, 1)

In [16]:
criterion = nn.BCEWithLogitsLoss()
lr = 0.005
optimizer = torch.optim.Adam(simple_model.parameters(), lr=lr)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
simple_model.to(device)

RNN(
  (i2e): Embedding(32835, 128)
  (e2h): Linear(in_features=128, out_features=128, bias=True)
  (h2h): Linear(in_features=128, out_features=128, bias=True)
  (h2o): Linear(in_features=128, out_features=1, bias=True)
)

In [18]:
num_epochs = 20
num_embeddings = simple_model.i2e.num_embeddings

losses = []
val_losses = []
for epoch in range(num_epochs):
    simple_model.train()
    for i, (texts, labels) in enumerate(tqdm(train_dataloader)):
        if torch.isnan(texts).any() or torch.isinf(texts).any():
            print(f"Found NaN or inf in texts at iteration {i}")
        if torch.isnan(labels).any() or torch.isinf(labels).any():
            print(f"Found NaN or inf in labels at iteration {i}")
        if (texts >= num_embeddings).any():
            print(f"Found out-of-range values in texts at iteration {i}")
        texts = texts.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        hidden = simple_model.initHidden().to(device)
        
        output, hidden = simple_model(texts, hidden)
        output = output.squeeze()  # Remove the extra dimension from the output

        loss = criterion(output, labels.float())  # Compute the loss
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    simple_model.eval()

    with torch.no_grad():
        for i, (texts, labels) in enumerate(tqdm(val_dataloader)):
            texts = texts.to(device)
            labels = labels.to(device)

            hidden = simple_model.initHidden().to(device)
            output, hidden = simple_model(texts, hidden)
            output = output.squeeze()

            val_loss = criterion(output, labels.float())
            val_losses.append(val_loss.item())

    print('Epoch [{}/{}], Validation Loss: {:.4f}'.format(epoch+1, num_epochs, val_loss.item()))

100%|██████████| 4/4 [00:00<00:00, 32.00it/s]


Epoch [1/20], Validation Loss: 0.7016


100%|██████████| 4/4 [00:00<00:00, 31.75it/s]


Epoch [2/20], Validation Loss: 0.6954


100%|██████████| 4/4 [00:00<00:00, 35.71it/s]


...

Epoch [18/20], Validation Loss: 0.6921


100%|██████████| 4/4 [00:00<00:00, 32.00it/s]


Epoch [19/20], Validation Loss: 0.6935


100%|██████████| 4/4 [00:00<00:00, 32.52it/s]

Epoch [20/20], Validation Loss: 0.6984


### Actividad 3

Implemente una red variante al RNN clásico, como un LSTM, GRU, Bi-LSTM, entre otros. Realice el entrenamiento del modelo con el mismo dataset. Realice un breve análisis sobre las diferencias en términos de los resultados, tiempo de ejecución y aspectos que le parezcan relevantes, entre la red implementada en la actividad previa y la red implementada en esta actividad.

In [ ]:
class VariantRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_size):
        super(VariantRNN, self).__init__()
        # TODO

In [ ]:
def train(model):
  pass

## Parte 2: Transformer

### Actividad 4

Responda las siguientes preguntas:



*   ¿Cuáles son los principales aspectos de las redes Transformer?
¿Qué desafíos específicos de las RNNs intenta superar la arquitectura de los Transformers?

    **Respuesta:**....

*   ¿Cuáles son las diferencias entre las capas cross-attention y self-attention?

    **Respuesta:**....

*   ¿Cuáles son las principales características del encoder y el decoder de una red Transformer?

    **Respuesta:**....

*   ¿En qué casos se aplica modelos de tipo encoder?. En qué casos se aplica modelos de tipo decoder?. ¿En qué casos se aplica modelos de tipo encoder decoder?. Responda y justifique su respuesta brevemente.

    **Respuesta:**....

### Actividad 5

En particular, la tarea será sobre generación de reseñas. Es decir que tendrán que implementar una red Transformer y entrenarla con el dataset IMDB para generar texto de reseñas de películas.

A medida que progresa el entrenamiento, muestre algunas instancias de salida del texto generado por el modelo. Para ello considere los puntos inicial, intermedio y final del entrenamiento. Realice un análisis sobre el texto generado y explique brevemente cómo el modelo es capaz de generar texto de reseñas.

Es importante que argumente su elección de los parámetros e hiperparámetros que vaya a utilizar, tales como: cantidad de capas, cantidad de neuronas por capa, función de activación, función de pérdida, etc. En particular, describa el método utilizado para la tokenización de los datos.

In [ ]:
class MyModel(nn.Module):
   def __init__(self):
        super().__init__()
        pass

In [ ]:
def train(model):
  pass

La siguiente celda es un ejemplo del entrenamiento de su modelo y de la generación de reseñas que realiza su modelo entrenado. En este caso se muestra la generación de 2000 tokens.

In [ ]:
model = MyModel()

train(model)

num_tokens = 2000

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=num_tokens)[0].tolist()))

0.224306 M parameters
step 0: train loss 5.3014, val loss 5.3014
step 100: train loss 2.7339, val loss 2.7166
step 200: train loss 2.5480, val loss 2.5588
step 300: train loss 2.4732, val loss 2.4783
step 400: train loss 2.3983, val loss 2.3997
step 500: train loss 2.3159, val loss 2.3250
step 600: train loss 2.2822, val loss 2.2755
step 700: train loss 2.2273, val loss 2.2159
step 800: train loss 2.1654, val loss 2.1702
step 900: train loss 2.1472, val loss 2.1538
step 1000: train loss 2.1127, val loss 2.1100
step 1100: train loss 2.0843, val loss 2.0968
step 1200: train loss 2.0706, val loss 2.0647
step 1300: train loss 2.0495, val loss 2.0576
step 1400: train loss 2.0166, val loss 2.0264
step 1500: train loss 2.0031, val loss 1.9983
step 1600: train loss 1.9816, val loss 1.9836
step 1700: train loss 1.9620, val loss 1.9739
step 1800: train loss 1.9528, val loss 1.9667
step 1900: train loss 1.9252, val loss 1.9404
step 2000: train loss 1.9206, val loss 1.9348
step 2100: train loss 1.

In [ ]:
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

Review: (TH Of E NOU'NYTN CE) No jealloa girl suwital. Unounder. I gon that HANEY SENK. SHEEG. All is helpinnes, that is job and AltohCRE: Fronk AGE A Sdenterns Review: Lusbe Allf Flaegray Ram-, high scist, butment was little nothim to yet. There been turn the good much, and thought be ver factuul to one yous heldined expy woutch be in creadity for Orché no culla real on monstanshing takeour ly grourie and some and panter and about my about crist? Well by infully. She Nebs and I wald they MTIE FOMD, Alexra TV Oh- hard, it those diffect, tries and Fene, more get yong trody or bad a torie writh this, you pay.<br /> /><br />I `00 a liude show ence have unding the tinfully. He right Combering Jet encesten with (looking, stakes) is a lost get it on in the film, but that stophiles it the bere I watch zitious that scene the `T.. Nonzeplay? The goals bad do - in clasks of his for than asks and to but. On bout, doire and relazing, the and the criseum and tract crimitly watchonde thip is the sur

### Actividad 6

A partir del modelo generativo de la anterior actividad, realice los ajustes necesarios al modelo para mejorar la calidad de las reseñas generadas. Se espera que el modelo sea capaz de generar reseñas de mejor calidad en comparación a la actividad previa. Para ello, investigue y agregue otros datasets públicos de reseñas de películas u otro tipo.